In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Twitter_Data.csv')

In [3]:
df.head(10)

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
5,kiya tho refresh maarkefir comment karo,0.0
6,surat women perform yagna seeks divine grace f...,0.0
7,this comes from cabinet which has scholars lik...,0.0
8,with upcoming election india saga going import...,1.0
9,gandhi was gay does modi,1.0


In [4]:
df[df.index==6]['clean_text']

6    surat women perform yagna seeks divine grace f...
Name: clean_text, dtype: object

In [5]:
df.category.unique()

array([-1.,  0.,  1., nan])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


### Fetching Harvard dictionary to get positive and negative words 

In [7]:
f = open('inqdict.txt')
HIDict = f.read()
HIDict = HIDict.splitlines()
HIDict[:20]

['Entryword Source Pos Neg Pstv Affil Ngtv Hostile Strng Power Weak Subm Actv Psv Pleasure Pain Arousal EMOT Feel Virtue Vice Ovrst Undrst Acad Doctr Econ* Exch ECON Exprs Legal Milit Polit* POLIT Relig Role COLL Work Ritual Intrel Race Kin* MALE Female Nonadlt HU ANI PLACE Social Region Route Aquatic Land Sky Object Tool Food Vehicle Bldgpt Natobj Bodypt Comnobj Comform COM Say Need Goal Try Means Ach Persist Complt Fail Natpro Begin Vary Change Incr Decr Finish Stay Rise Move Exert Fetch Travel Fall Think Know Causal Ought Percv Comp Eval EVAL Solve Abs* ABS Qual Quan NUMB ORD CARD FREQ DIST Time* TIME Space POS DIM Dimn Rel COLOR Self Our You Name Yes No Negate Intrj IAV DAV SV IPadj IndAdj POWGAIN POWLOSS POWENDS POWAREN POWCON POWCOOP POWAPT POWPT POWDOCT POWAUTH POWOTH POWTOT RCTETH RCTREL RCTGAIN RCTLOSS RCTENDS RCTTOT RSPGAIN RSPLOSS RSPOTH RSPTOT AFFGAIN AFFLOSS AFFPT AFFOTH AFFTOT WLTPT WLTTRAN WLTOTH WLTTOT WLBGAIN WLBLOSS WLBPHYS WLBPSYC WLBPT WLBTOT ENLGAIN ENLLOSS ENLENDS

In [8]:
poswords = [j for j in HIDict if "Pos" in j]  #using a list comprehension
poswords = [j.split()[0] for j in poswords]
poswords = [j.split("#")[0] for j in poswords]
poswords = set(poswords)
poswords = [j.lower() for j in poswords]
print(poswords[:20])
print(len(poswords))

['equitable', 'popularity', 'solution', 'compassion', 'brotherly', 'suitable', 'rich', 'adhesive', 'glamorous', 'save', 'strut', 'baptism', 'eager', 'fearless', 'conducive', 'heartily', 'humor', 'sparkle', 'adjustable', 'candid']
1647


In [9]:
#Extract all the lines that contain the Neg tag
negwords = [j for j in HIDict if "Neg" in j]  #using a list comprehension
negwords = [j.split()[0] for j in negwords]
negwords = [j.split("#")[0] for j in negwords]
negwords = set(negwords)
negwords = [j.lower() for j in negwords]
print(negwords[:20])
print(len(negwords))

['rather', 'abhor', 'straggler', 'crumble', 'sprain', 'disclaim', 'envious', 'confess', 'severity', 'spite', 'noise', 'straggle', 'accost', 'resignation', 'avoid', 'meaningless', 'cross', 'shame', 'vagrant', 'conspiracy']
2121


In [10]:
df1=df.copy()

In [11]:
df1.isna().sum()

clean_text    4
category      7
dtype: int64

### Function to fill the NA values in clean text column according to values present in category column

In [12]:
def filling_na(df):
    if pd.isna(df['clean_text']):
        if df['category']==0.0:
            df['clean_text']='neutral'
        elif df['category']==1.0:
            df['clean_text']='positive'
        else:
            df['clean_text']='negative'
    return df

In [13]:
df2 = df1.apply(filling_na, axis=1)

In [14]:
df2[pd.isna(df2['category'])]

,clean_text,category
130448,the foundation stone northeast gas grid inaugu...,NaN
155642,dear terrorists you can run but you cant hide ...,NaN
155698,offense the best defence with mission shakti m...,NaN
155770,have always heard politicians backing out thei...,NaN
158693,modi government plans felicitate the faceless ...,NaN
159442,chidambaram gives praises modinomics,NaN
160559,the reason why modi contested from seats 2014 ...,NaN


Using the NLTK package for the stopwords

In [15]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vignesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vignesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Function to remove punctuation, numbers etc

In [16]:
# Functions to remove Punctuations, Numbers and stop words 

import string
def removePuncStr(s):
    for c in string.punctuation:
        s = s.replace(c," ")
    return s

def removePunc(text_array):
    return [removePuncStr(h) for h in text_array]


def removeNumbersStr(s):
    for c in range(10):
        n = str(c)
        s = s.replace(n," ")
    return s

def removeNumbers(text_array):
    return [removeNumbersStr(h) for h in text_array]

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def stopText(text_array):
    stop_words = set(stopwords.words('english'))
    stopped_text = []
    for h in text_array:
        words = word_tokenize(h)
        h2 = ''
        for w in words:
            if w.lower() not in stop_words:
                h2 = h2 + ' ' + w
        stopped_text.append(h2)
    return stopped_text

In [17]:
df2['clean_text2'] = df2.clean_text
df2['clean_text2'] = removePunc(df2.clean_text)
df2['clean_text2'] = removeNumbers(df2.clean_text)
df2['clean_text2'] = stopText(df2.clean_text)

In [18]:
df2.head()

,clean_text,category,clean_text2
0,when modi promised “minimum government maximum...,-1.0,modi promised “ minimum government maximum go...
1,talk all the nonsense and continue all the dra...,0.0,talk nonsense continue drama vote modi
2,what did just say vote for modi welcome bjp t...,1.0,say vote modi welcome bjp told rahul main cam...
3,asking his supporters prefix chowkidar their n...,1.0,asking supporters prefix chowkidar names modi...
4,answer who among these the most powerful world...,1.0,answer among powerful world leader today trum...


### Stemming the text to reduce it to its root form

In [19]:
# To stem the text

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

def stemText(text_array):
    stemmed_text = []
    for h in text_array:
        words = word_tokenize(h)
        h2 = ''
        for w in words:
            h2 = h2 + ' ' + PorterStemmer().stem(w)
        stemmed_text.append(h2)
    return stemmed_text
    

In [20]:
df2['clean_stemmed'] = 0

Function to convert the sentences into list of words

In [21]:
def Convert(string):
    li = list(string.split(" "))
    return li

for i in range(len(df2)):
    df2.clean_stemmed[i] = Convert(df2.clean_text2[i])

<ipython-input-21-2e524ca1e0c0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.clean_stemmed[i] = Convert(df2.clean_text2[i])
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
poswords = stemText(poswords)
negwords = stemText(negwords)

for i in range(len(poswords)):
    poswords[i] = poswords[i].strip(" ")

for i in range(len(negwords)):
    negwords[i] = negwords[i].strip(" ")

### Finding the number of positive and negative words in each stemmed collection

In [24]:
df2['Pos_words'] = 0
df2['Neg_words'] = 0

for i in range(len(df2)):
    df2['Pos_words'][i] = len(set(df2.clean_stemmed[i]).intersection(set(poswords)))

for i in range(len(df)):
    df2['Neg_words'][i] = len(set(df2.clean_stemmed[i]).intersection(set(negwords)))

<ipython-input-24-c39d396efe1c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Pos_words'][i] = len(set(df2.clean_stemmed[i]).intersection(set(poswords)))
<ipython-input-24-c39d396efe1c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Neg_words'][i] = len(set(df2.clean_stemmed[i]).intersection(set(negwords)))


In [25]:
df2.head()

,clean_text,category,clean_text2,clean_stemmed,Pos_words,Neg_words
0,when modi promised “minimum government maximum...,-1.0,modi promised “ minimum government maximum go...,"[, modi, promised, “, minimum, government, max...",1,3
1,talk all the nonsense and continue all the dra...,0.0,talk nonsense continue drama vote modi,"[, talk, nonsense, continue, drama, vote, modi]",0,0
2,what did just say vote for modi welcome bjp t...,1.0,say vote modi welcome bjp told rahul main cam...,"[, say, vote, modi, welcome, bjp, told, rahul,...",2,0
3,asking his supporters prefix chowkidar their n...,1.0,asking supporters prefix chowkidar names modi...,"[, asking, supporters, prefix, chowkidar, name...",2,1
4,answer who among these the most powerful world...,1.0,answer among powerful world leader today trum...,"[, answer, among, powerful, world, leader, tod...",0,0


In [26]:
df2.drop(columns=['clean_text','clean_text2'], inplace=True)

In [27]:
df2.head()

,category,clean_stemmed,Pos_words,Neg_words
0,-1.0,"[, modi, promised, “, minimum, government, max...",1,3
1,0.0,"[, talk, nonsense, continue, drama, vote, modi]",0,0
2,1.0,"[, say, vote, modi, welcome, bjp, told, rahul,...",2,0
3,1.0,"[, asking, supporters, prefix, chowkidar, name...",2,1
4,1.0,"[, answer, among, powerful, world, leader, tod...",0,0


In [28]:
df2.drop(columns='clean_stemmed', inplace=True)

### There are 7 records having NaN values in category. These are seperated to form the test data using model prediction

In [29]:
df_test=df2[df2.category.isna()]

In [30]:
df2=df2[~pd.isnull(df2['category'])]

### Using number of positive and negative words as the only columns for predicting category as we want numerical values in columns . These two columns are derived from clean text column.  

In [31]:
df2

,category,Pos_words,Neg_words
0,-1.0,1,3
1,0.0,0,0
2,1.0,2,0
3,1.0,2,1
4,1.0,0,0
...,...,...,...
162975,-1.0,0,2
162976,-1.0,1,2
162977,0.0,0,0
162978,0.0,0,0


### Splitting data into train and test in 80: 20 ratio

In [32]:
# Train test split 
from sklearn.model_selection import train_test_split
X1 = df2.drop(['category'],axis=1)
Y1 = df2.category



X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1,test_size=0.2,random_state = 42)

### Modeling

In [33]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X1_train_scaled = scaler.fit_transform(X1_train)
X1_test_scaled = scaler.transform(X1_test)

In [36]:
%%time
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier(n_estimators=500, max_depth=7, random_state=3)
model1.fit(X1_train_scaled,Y1_train)

CPU times: user 10.5 s, sys: 249 ms, total: 10.8 s
Wall time: 10.9 s


RandomForestClassifier(max_depth=7, n_estimators=500, random_state=3)

In [37]:
Y1_pred = model1.predict(X1_test_scaled)
Y1_pred.shape

(32595,)

In [39]:
from sklearn.metrics import accuracy_score

accuracy_score(Y1_pred, Y1_test)

0.5499923301119803

In [43]:
df_test1=df_test.drop(columns='category')

### Predicting the test values where category values were NA

In [44]:
model1.predict(df_test1)

array([1., 1., 1., 1., 1., 1., 1.])